In [15]:
# Configuration

from datetime import datetime, date
from google.cloud import bigquery
import os
import pandas as pd
file = '/Users/blakeolsen/Projects/grid-analysis-engine/data/chicago_2019.csv'
location = 'Chicago'
table_name = 'grid-analysis-engine.noaa_org.hourly_data'

In [22]:
# ETL

def float_or_none(value):
    try:
        return float(value)
    except Exception:
        print(f"skipping {value}")
        return None
    
denormalized = pd.read_csv(file)
normalized = pd.DataFrame(data={
    'TS': denormalized['DATE'].apply(datetime.fromisoformat),
    'WIND': denormalized['HourlyWindSpeed'].apply(float_or_none),
    'TEMP_CELCIUS': denormalized['HourlyDryBulbTemperature'].apply(float_or_none)
})
timeseries = normalized.drop_duplicates(subset=['TS']).set_index('TS').resample('H').interpolate().reset_index()
timeseries['LOCATION'] = location

skipping 79s
skipping 79s


/Users/blakeolsen/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (30,31,33,43,69,71,75,88,89) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [41]:
# UPLOAD to BIGQUERY
# dumb logic to stop if either the ends are included

bq = bigquery.Client()
query_results = bq.query(
    f"""
    SELECT *
    FROM grid-analysis-engine.noaa_org.hourly_data
    WHERE
      LOCATION = '{location}' AND (
      TS = TIMESTAMP("{timeseries['TS'][0].isoformat(sep=' ')}") OR
      TS = TIMESTAMP("{list(timeseries.iloc[[-1]]['TS'])[0].isoformat(sep=" ")}")
      )
    """,
    job_config = bigquery.QueryJobConfig(),
    
)
if query_results.result().total_rows > 0:
    print(f"Overlapping existing data")
else:
    table = bq.get_table(table_name)
    print(f"Loading data to {table_name}")
    #bq.load_table_from_dataframe(timeseries, table)

Overlapping existing data
